In [58]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F

In [59]:
prepped = pd.read_csv('/Users/dave/code/old-betsy/data/full.csv')
prepped

,callsign,de_pfx,de_cont,freq,band,dx,dx_pfx,dx_cont,mode,db,...,P. Tunguska_A_index,P. Tunguska_K_index,Petropavlovsk_A_index,Petropavlovsk_K_index,Planetary(estimated Ap)_A_index,Planetary(estimated Ap)_K_index,Tiksi Bay_A_index,Tiksi Bay_K_index,Wingst_A_index,Wingst_K_index
0,MW0MUT,GW,EU,3521.0,80m,DL5NDH,DL,EU,CQ,25.0,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
1,DK0TE,DL,EU,3521.0,80m,DL5NDH,DL,EU,CQ,32.0,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
2,DK0TE,DL,EU,3518.0,80m,LB2TB,LA,EU,CQ,18.0,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
3,DR4W,DL,EU,3521.0,80m,DL5NDH,DL,EU,CQ,29.0,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
4,UA4M,UA,EU,3521.0,80m,DL5NDH,DL,EU,CQ,20.0,...,-1.0,-1.0,-1.0,-1.0,11.0,11.0,-1.0,-1.0,16.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681611,DL0PF,DL,EU,3560.0,80m,EG4CIB,EA,EU,CQ,5.0,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681612,SA1CCQ,SM,EU,3550.0,80m,DO6IH,DL,EU,CQ,14.0,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681613,WC2L,K,NaN,7055.0,40m,UE3BXG,UA,EU,CQ,7.0,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0
681614,W2NAF,K,NaN,10120.0,30m,AE0EE,K,NaN,CQ,16.0,...,-1.0,-1.0,-1.0,-1.0,11.0,3.0,-1.0,-1.0,16.0,4.0


In [60]:
essential_columns = prepped.drop(columns=['freq', 'speed', 'callsign', 'dx', 'de_cont', 'dx_cont', "de_pfx", "dx_pfx", "callsign_lat", "callsign_lon", "dx_lat", "dx_lon"], axis=1)

In [61]:
na_removed=essential_columns.copy()
na_removed = na_removed.dropna(subset=['callsign_grid', 'dx_grid'])
len(na_removed)


356800

In [62]:
date_exploded = na_removed.copy()
date_exploded['year'] = pd.to_datetime(date_exploded['date']).dt.year
date_exploded['month'] = pd.to_datetime(date_exploded['date']).dt.month
date_exploded['day'] = pd.to_datetime(date_exploded['date']).dt.day
date_exploded['hour'] = pd.to_datetime(date_exploded['date']).dt.hour
date_exploded['minute'] = pd.to_datetime(date_exploded['date']).dt.minute
date_exploded['second'] = pd.to_datetime(date_exploded['date']).dt.second
date_exploded = date_exploded.drop(columns='date')

In [63]:
encoding_dictionary = {}

for key in date_exploded.columns:
    encoding_dictionary[key] = date_exploded[key].unique()
    encoding_dictionary[key] = sorted(encoding_dictionary[key])

encoding_dictionary['grid'] = encoding_dictionary['callsign_grid'] + encoding_dictionary['dx_grid']

In [64]:
encoded = date_exploded.copy()
encoded['band'] = encoded['band'].map(lambda x: encoding_dictionary['band'].index(x))
encoded['mode'] = encoded['mode'].map(lambda x: encoding_dictionary['mode'].index(x))
encoded['callsign_grid'] = encoded['callsign_grid'].map(lambda x: encoding_dictionary['grid'].index(x))
encoded['dx_grid'] = encoded['dx_grid'].map(lambda x: encoding_dictionary['grid'].index(x))
encoded['tx_mode'] = encoded['tx_mode'].map(lambda x: encoding_dictionary['tx_mode'].index(x))
encoded

,band,mode,db,tx_mode,callsign_grid,dx_grid,Beijing_A_index,Beijing_K_index,Belsk_A_index,Belsk_K_index,...,Tiksi Bay_A_index,Tiksi Bay_K_index,Wingst_A_index,Wingst_K_index,year,month,day,hour,minute,second
0,12,1,25.0,0,56,1724,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,16.0,2022,1,1,0,0,0
1,12,1,32.0,0,71,1724,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,16.0,2022,1,1,0,0,0
2,12,1,18.0,0,71,1775,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,16.0,2022,1,1,0,0,0
3,12,1,29.0,0,78,1724,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,16.0,2022,1,1,0,0,0
4,12,1,20.0,0,110,1724,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,16.0,2022,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681577,12,1,32.0,0,45,38,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,4.0,2022,1,1,23,59,42
681599,5,3,10.0,0,34,477,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,4.0,2022,1,1,23,59,51
681605,5,3,22.0,0,116,1992,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,4.0,2022,1,1,23,59,54
681614,7,1,16.0,0,43,417,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,16.0,4.0,2022,1,1,23,59,59


In [65]:
encoded['db'].unique()

array([ 25.,  32.,  18.,  29.,  20.,  26.,  11.,  14.,  28.,  13.,  16.,
        30.,  39.,  12.,  10.,  27.,   7.,  22.,   4.,  15.,   8.,  17.,
        24.,  41.,   6.,   5.,  31.,   9.,   3.,  19.,  23.,  35.,  38.,
        34.,  21.,  47.,  36.,  46.,  33.,  42.,  48.,  54.,  43.,  37.,
        40.,  51.,   1.,  45.,  44.,   2.,  52.,  49.,  56.,  53.,  85.,
        92.,  64.,  63.,  62.,  55.,  58., 117.,  50.,  60.,   0.,  86.,
        61.,  82.,  57.,  59.,  65.,  -1.,  67.,  69.,  66.,  68.,  88.,
        91.,  90.,  84.,  76.,  80.,  77.,  97.,  74.,  83.,  75.])

In [66]:
torch_tensor = torch.tensor(encoded['band'].values)


In [55]:
msk = np.random.rand(len(fe_3)) < 0.8

NameError: name 'fe_3' is not defined

In [ ]:
train = fe_1[msk]
test = fe_1[~msk]

In [ ]:
len(train) + len(test) == len(fe_1)

False

In [ ]:
train.to_csv("data/train.csv", mode="w")
test.to_csv("data/test.csv", mode="w")